# Lab 6.1: Fick's Law Gas Exchange Calculator
## Chapter 6: The Oxygen Revolution - Respiratory Systems

### 🎯 Learning Objectives
By completing this interactive lab, you will:
- Apply Fick's Law of Diffusion to analyze respiratory systems
- Calculate and compare gas exchange rates across different vertebrate groups
- Understand how surface area, membrane thickness, and concentration gradients affect efficiency
- Predict respiratory performance based on anatomical parameters
- Connect physical principles to evolutionary adaptations

### 📖 Connection to Chapter 6
This lab explores the concepts from **Section 6.1: The Gas Exchange Imperative**, where we learned that Fick's Law governs all vertebrate gas exchange:

**Rate of Diffusion = (Surface Area × Concentration Gradient × Diffusion Coefficient) / Membrane Thickness**

This simple equation explains why gills have enormous folded surfaces, why exchange membranes are incredibly thin (0.1-0.5 micrometers), and why counter-current flow is so effective.

In [12]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from ipywidgets import interact, interactive, FloatSlider, Dropdown, VBox, HBox, Button, Output, HTML
import io
import base64
from IPython.display import display, Image as IPImage

print("✓ All libraries loaded successfully!")
print("Ready to explore respiratory gas exchange...")

✓ All libraries loaded successfully!
Ready to explore respiratory gas exchange...


In [13]:
# === GOOGLE COLAB SETUP - RUN THIS FIRST ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled for Google Colab")
except:
    print("✓ Running outside Colab")

# Then import other libraries...
import numpy as np
import pandas as pd
from ipywidgets import interact, FloatSlider, Dropdown, Button

✓ Widgets enabled for Google Colab


## Part 1: Understanding Fick's Law

### The Physics of Gas Exchange

Every vertebrate faces the same challenge: getting oxygen into cells and carbon dioxide out. Fick's Law tells us exactly what determines the rate of this process:

$$\text{Gas Exchange Rate} = \frac{A \times \Delta P \times D}{T}$$

Where:
- **A** = Surface area available for exchange (m²)
- **ΔP** = Concentration gradient (partial pressure difference, mmHg)
- **D** = Diffusion coefficient (depends on gas and temperature)
- **T** = Membrane thickness (micrometers)

### What This Means for Evolution

To increase gas exchange, respiratory systems can:
1. **Maximize surface area** (folding, branching) ↑ A
2. **Minimize thickness** (flatten cells, reduce barrier) ↓ T  
3. **Maintain gradients** (ventilation, counter-current flow) ↑ ΔP
4. **Optimize temperature** (affects D, but limited by physiology)

Let's see how different vertebrates have solved this challenge!

In [14]:
# Define the Fick's Law calculation function
def calculate_gas_exchange(surface_area, gradient, thickness, temperature=37, gas='O2'):
    """
    Calculate gas exchange rate using Fick's Law

    Parameters:
    -----------
    surface_area : float
        Exchange surface area in m²
    gradient : float
        Partial pressure difference in mmHg
    thickness : float
        Membrane thickness in micrometers
    temperature : float
        Temperature in Celsius (affects diffusion coefficient)
    gas : str
        Either 'O2' or 'CO2'

    Returns:
    --------
    float : Gas exchange rate in ml/min
    """
    # Diffusion coefficients (simplified, temperature-dependent)
    # Units: ml/(min·m²·mmHg·μm)
    base_D_O2 = 0.00023  # oxygen at 37°C
    base_D_CO2 = 0.00046  # CO2 diffuses ~2x faster

    # Temperature correction (Q10 = 1.3 for diffusion)
    temp_factor = 1.3 ** ((temperature - 37) / 10)

    D = (base_D_O2 if gas == 'O2' else base_D_CO2) * temp_factor

    # Fick's Law: Rate = (A × ΔP × D) / T
    # Convert thickness from μm to m for dimensional consistency
    thickness_m = thickness * 1e-6

    exchange_rate = (surface_area * gradient * D) / thickness

    return exchange_rate

# Test the function
test_rate = calculate_gas_exchange(surface_area=1.0, gradient=60, thickness=0.5)
print(f"Test calculation: {test_rate:.2f} ml O₂/min")
print("\n✓ Fick's Law calculator ready!")

Test calculation: 0.03 ml O₂/min

✓ Fick's Law calculator ready!


## Part 2: Respiratory Structure Presets

Let's compare real respiratory systems from the chapter. Each preset represents actual anatomical data:

In [15]:
# Define preset respiratory structures based on Chapter 6 data
respiratory_presets = {
    'Great White Shark Gill': {
        'surface_area': 7.0,  # m² (6-8 m² from chapter)
        'thickness': 0.2,  # μm (very thin for efficiency)
        'gradient': 45,  # mmHg (counter-current maintains high gradient)
        'description': 'High-performance gill system with counter-current flow. '
                      'Ram ventilation during continuous swimming. '
                      '60-70% oxygen extraction efficiency.',
        'group': 'Fish',
        'efficiency': 65
    },
    'Human Lungs (Alveoli)': {
        'surface_area': 70.0,  # m² (typical adult)
        'thickness': 0.5,  # μm (alveolar-capillary membrane)
        'gradient': 60,  # mmHg (PaO₂ ~100, PvO₂ ~40)
        'description': 'Mammalian alveolar system with enormous surface area. '
                      'Approx. 300 million alveoli creating ~70 m² of exchange surface.',
        'group': 'Mammal',
        'efficiency': 25
    },
    'Bar-Headed Goose Air Sacs': {
        'surface_area': 0.8,  # m² (smaller bird, but efficient system)
        'thickness': 0.15,  # μm (extremely thin for high altitude)
        'gradient': 70,  # mmHg (unidirectional flow maintains high gradient)
        'description': 'Ultimate respiratory performance. Unidirectional airflow '
                      'through parabronchi. Flights above 8,000m. Enhanced '
                      'hemoglobin oxygen affinity.',
        'group': 'Bird',
        'efficiency': 90
    },
    'Indian Bullfrog Lung': {
        'surface_area': 0.05,  # m² (simple sac-like structure)
        'thickness': 2.0,  # μm (thicker than mammals/birds)
        'gradient': 50,  # mmHg
        'description': 'Simple sac-like amphibian lung with moderate internal folding. '
                      'Supplemented by cutaneous respiration (30-60% of gas exchange '
                      'through skin).',
        'group': 'Amphibian',
        'efficiency': 20
    },
    'Python Lung': {
        'surface_area': 0.3,  # m² (multi-chambered reptilian lung)
        'thickness': 0.8,  # μm
        'gradient': 55,  # mmHg
        'description': 'Multi-chambered reptilian lung with improved complexity '
                      'over amphibians. Internal septa increase surface area. '
                      'More efficient than amphibian lungs.',
        'group': 'Reptile',
        'efficiency': 30
    },
    'Lungfish Dual System': {
        'surface_area': 0.12,  # m² (combined gill and lung)
        'thickness': 1.5,  # μm
        'gradient': 40,  # mmHg
        'description': 'Transitional form maintaining both gills and primitive lungs. '
                      'Can switch between aquatic and aerial respiration. '
                      'Demonstrates evolutionary transition to land.',
        'group': 'Fish',
        'efficiency': 35
    }
}

print("Respiratory System Presets Loaded:")
print("=" * 50)
for name, data in respiratory_presets.items():
    print(f"\n{name} ({data['group']})")
    print(f"  Surface Area: {data['surface_area']:.2f} m²")
    print(f"  Membrane Thickness: {data['thickness']:.2f} μm")
    print(f"  Extraction Efficiency: {data['efficiency']}%")

Respiratory System Presets Loaded:

Great White Shark Gill (Fish)
  Surface Area: 7.00 m²
  Membrane Thickness: 0.20 μm
  Extraction Efficiency: 65%

Human Lungs (Alveoli) (Mammal)
  Surface Area: 70.00 m²
  Membrane Thickness: 0.50 μm
  Extraction Efficiency: 25%

Bar-Headed Goose Air Sacs (Bird)
  Surface Area: 0.80 m²
  Membrane Thickness: 0.15 μm
  Extraction Efficiency: 90%

Indian Bullfrog Lung (Amphibian)
  Surface Area: 0.05 m²
  Membrane Thickness: 2.00 μm
  Extraction Efficiency: 20%

Python Lung (Reptile)
  Surface Area: 0.30 m²
  Membrane Thickness: 0.80 μm
  Extraction Efficiency: 30%

Lungfish Dual System (Fish)
  Surface Area: 0.12 m²
  Membrane Thickness: 1.50 μm
  Extraction Efficiency: 35%


## Part 3: Interactive Gas Exchange Calculator

Now let's explore how changing each parameter affects respiratory performance. You can either:
1. **Load a preset** to see real animal respiratory systems
2. **Adjust sliders** to design your own respiratory system
3. **Compare** different solutions to the same challenge

In [16]:
# Create interactive calculator with visualization
class GasExchangeCalculator:
    def __init__(self):
        self.current_data = None
        self.comparison_data = []

    def calculate_and_display(self, preset, surface_area, thickness, gradient, temperature, gas_type):
        """
        Calculate gas exchange and create visualization
        """
        # If preset is selected, load those values
        if preset != 'Custom':
            preset_data = respiratory_presets[preset]
            surface_area = preset_data['surface_area']
            thickness = preset_data['thickness']
            gradient = preset_data['gradient']

        # Calculate exchange rate
        exchange_rate = calculate_gas_exchange(
            surface_area, gradient, thickness, temperature, gas_type
        )

        # Store current configuration
        self.current_data = {
            'name': preset if preset != 'Custom' else 'Custom Design',
            'surface_area': surface_area,
            'thickness': thickness,
            'gradient': gradient,
            'temperature': temperature,
            'gas_type': gas_type,
            'exchange_rate': exchange_rate
        }

        # Create visualizations
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Gas Exchange Rate',
                'Parameter Contributions',
                'Membrane Cross-Section',
                'Fick\'s Law Components'
            ),
            specs=[
                [{'type': 'indicator'}, {'type': 'bar'}],
                [{'type': 'scatter'}, {'type': 'bar'}]
            ],
            vertical_spacing=0.15,
            horizontal_spacing=0.12
        )

        # 1. Main exchange rate indicator
        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=exchange_rate,
                title={'text': f"{gas_type} Exchange Rate<br>(ml/min)"},
                number={'suffix': ' ml/min', 'font': {'size': 40}},
                domain={'x': [0, 1], 'y': [0, 1]}
            ),
            row=1, col=1
        )

        # 2. Parameter contributions (showing relative impact)
        # Higher values = more contribution to exchange rate
        param_contributions = {
            'Surface Area': surface_area / 10,  # Normalize for display
            'Gradient': gradient / 10,
            'Thinness': 5.0 / thickness  # Inverse of thickness
        }

        fig.add_trace(
            go.Bar(
                x=list(param_contributions.keys()),
                y=list(param_contributions.values()),
                marker_color=['#2E86AB', '#A23B72', '#F18F01'],
                text=[f"{v:.2f}" for v in param_contributions.values()],
                textposition='auto'
            ),
            row=1, col=2
        )

        # 3. Membrane cross-section visualization
        # Show how oxygen travels across the barrier
        x_membrane = np.linspace(0, thickness, 100)
        # Concentration decreases linearly across membrane (Fick's assumption)
        concentration = gradient * (1 - x_membrane/thickness)

        fig.add_trace(
            go.Scatter(
                x=x_membrane,
                y=concentration,
                fill='tozeroy',
                name='O₂ Gradient',
                line=dict(color='#0077B6', width=3),
                fillcolor='rgba(0, 119, 182, 0.3)'
            ),
            row=2, col=1
        )

        # 4. Fick's Law component values
        components = {
            f'Area<br>({surface_area:.1f} m²)': surface_area,
            f'Gradient<br>({gradient} mmHg)': gradient,
            f'Thinness<br>({thickness:.2f} μm)': 10/thickness  # Show inverse for positive correlation
        }

        fig.add_trace(
            go.Bar(
                x=list(components.keys()),
                y=list(components.values()),
                marker_color=['#06A77D', '#D4526E', '#F4D35E'],
                text=[f"{v:.2f}" for v in components.values()],
                textposition='auto'
            ),
            row=2, col=2
        )

        # Update layout
        fig.update_xaxes(title_text="Membrane Thickness (μm)", row=2, col=1)
        fig.update_yaxes(title_text="O₂ Concentration (mmHg)", row=2, col=1)
        fig.update_yaxes(title_text="Relative Contribution", row=1, col=2)
        fig.update_yaxes(title_text="Parameter Value", row=2, col=2)

        fig.update_layout(
            height=700,
            showlegend=False,
            title_text=f"<b>Respiratory System Analysis: {self.current_data['name']}</b>",
            title_x=0.5
        )

        # Display results
        print("\n" + "="*70)
        print(f"RESPIRATORY SYSTEM: {self.current_data['name']}")
        print("="*70)
        print(f"\nGas Exchange Rate: {exchange_rate:.2f} ml {gas_type}/min")
        print(f"\nParameters:")
        print(f"  • Surface Area: {surface_area:.2f} m²")
        print(f"  • Membrane Thickness: {thickness:.2f} μm")
        print(f"  • Concentration Gradient: {gradient} mmHg")
        print(f"  • Temperature: {temperature}°C")

        if preset != 'Custom':
            print(f"\nDescription: {respiratory_presets[preset]['description']}")
            print(f"Extraction Efficiency: {respiratory_presets[preset]['efficiency']}%")

        print("\n" + "="*70)

        fig.show()

    def add_to_comparison(self):
        """Add current configuration to comparison list"""
        if self.current_data:
            self.comparison_data.append(self.current_data.copy())
            print(f"✓ Added '{self.current_data['name']}' to comparison (Total: {len(self.comparison_data)})")

    def show_comparison(self):
        """Display comparison of multiple respiratory systems"""
        if len(self.comparison_data) < 2:
            print("⚠ Add at least 2 systems to compare!")
            return

        # Create comparison visualization
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Exchange Rate Comparison', 'Parameter Comparison'),
            specs=[[{'type': 'bar'}, {'type': 'bar'}]]
        )

        names = [d['name'] for d in self.comparison_data]
        rates = [d['exchange_rate'] for d in self.comparison_data]

        # Exchange rate comparison
        fig.add_trace(
            go.Bar(
                x=names,
                y=rates,
                marker_color='#2E86AB',
                text=[f"{r:.1f}" for r in rates],
                textposition='auto',
                name='Exchange Rate'
            ),
            row=1, col=1
        )

        # Parameter comparison (grouped bar chart)
        for param in ['surface_area', 'gradient']:
            values = [d[param] for d in self.comparison_data]
            fig.add_trace(
                go.Bar(
                    x=names,
                    y=values,
                    name=param.replace('_', ' ').title()
                ),
                row=1, col=2
            )

        fig.update_layout(
            height=500,
            title_text="<b>Respiratory System Comparison</b>",
            title_x=0.5,
            barmode='group'
        )

        fig.update_yaxes(title_text="ml O₂/min", row=1, col=1)
        fig.update_yaxes(title_text="Parameter Value", row=1, col=2)

        fig.show()

        # Print summary table
        df = pd.DataFrame(self.comparison_data)
        display(df[['name', 'exchange_rate', 'surface_area', 'thickness', 'gradient']])

    def clear_comparison(self):
        """Clear comparison data"""
        self.comparison_data = []
        print("✓ Comparison data cleared")

# Create calculator instance
calculator = GasExchangeCalculator()

print("\n✓ Interactive calculator ready!")
print("Run the next cell to start exploring...")


✓ Interactive calculator ready!
Run the next cell to start exploring...


In [17]:
# Interactive widget interface
preset_dropdown = Dropdown(
    options=['Custom'] + list(respiratory_presets.keys()),
    value='Custom',
    description='Preset:',
    style={'description_width': '120px'}
)

surface_slider = FloatSlider(
    value=1.0,
    min=0.01,
    max=100.0,
    step=0.1,
    description='Surface Area (m²):',
    style={'description_width': '150px'},
    readout_format='.2f'
)

thickness_slider = FloatSlider(
    value=0.5,
    min=0.1,
    max=5.0,
    step=0.1,
    description='Thickness (μm):',
    style={'description_width': '150px'},
    readout_format='.2f'
)

gradient_slider = FloatSlider(
    value=60,
    min=10,
    max=100,
    step=5,
    description='Gradient (mmHg):',
    style={'description_width': '150px'},
    readout_format='.0f'
)

temp_slider = FloatSlider(
    value=37,
    min=0,
    max=45,
    step=1,
    description='Temperature (°C):',
    style={'description_width': '150px'},
    readout_format='.0f'
)

gas_dropdown = Dropdown(
    options=['O2', 'CO2'],
    value='O2',
    description='Gas Type:',
    style={'description_width': '120px'}
)

# Create interactive output
output = interactive(
    calculator.calculate_and_display,
    preset=preset_dropdown,
    surface_area=surface_slider,
    thickness=thickness_slider,
    gradient=gradient_slider,
    temperature=temp_slider,
    gas_type=gas_dropdown
)

# Add comparison buttons
add_button = Button(
    description='Add to Comparison',
    button_style='success',
    icon='plus'
)
add_button.on_click(lambda b: calculator.add_to_comparison())

compare_button = Button(
    description='Show Comparison',
    button_style='info',
    icon='bar-chart'
)
compare_button.on_click(lambda b: calculator.show_comparison())

clear_button = Button(
    description='Clear Comparison',
    button_style='warning',
    icon='trash'
)
clear_button.on_click(lambda b: calculator.clear_comparison())

button_box = HBox([add_button, compare_button, clear_button])

# Display interface
display(HTML("<h3>🔬 Interactive Fick's Law Calculator</h3>"))
display(HTML("<p><i>Select a preset or adjust parameters to explore gas exchange rates</i></p>"))
display(output)
display(HTML("<br><h4>Comparison Tools:</h4>"))
display(button_box)

HTML(value="<h3>🔬 Interactive Fick's Law Calculator</h3>")

HTML(value='<p><i>Select a preset or adjust parameters to explore gas exchange rates</i></p>')

interactive(children=(Dropdown(description='Preset:', options=('Custom', 'Great White Shark Gill', 'Human Lung…

HTML(value='<br><h4>Comparison Tools:</h4>')

## Part 4: Metabolic Scaling Analysis

### Why Size Matters

From the chapter, we learned that oxygen consumption scales with body mass according to:

$$VO_2 \propto M^{0.75}$$

This means larger animals need less oxygen per gram of body mass, but still need enormous total exchange capacity. Let's see how respiratory systems scale to meet these demands.

In [18]:
# Metabolic scaling analysis
def calculate_oxygen_demand(body_mass_kg, activity_level='resting'):
    """
    Calculate oxygen consumption based on body mass and activity

    Parameters:
    -----------
    body_mass_kg : float
        Body mass in kilograms
    activity_level : str
        'resting', 'moderate', or 'intense'

    Returns:
    --------
    float : Oxygen consumption in ml O₂/min
    """
    # Basal metabolic rate: VO2 = 10 × M^0.75 (ml/min)
    basal_vo2 = 10 * (body_mass_kg ** 0.75)

    # Activity multipliers
    multipliers = {
        'resting': 1.0,
        'moderate': 5.0,
        'intense': 15.0
    }

    return basal_vo2 * multipliers[activity_level]

def predict_required_surface_area(body_mass_kg, activity_level='resting',
                                  thickness=0.5, gradient=60):
    """
    Predict minimum surface area needed for given body mass and activity
    """
    # Calculate oxygen demand
    oxygen_demand = calculate_oxygen_demand(body_mass_kg, activity_level)

    # Rearrange Fick's Law to solve for required surface area
    # A = (Rate × T) / (ΔP × D)
    D = 0.00023  # diffusion coefficient
    required_area = (oxygen_demand * thickness) / (gradient * D)

    return oxygen_demand, required_area

# Create scaling visualization
body_masses = np.logspace(-2, 4, 50)  # 0.01 kg to 10,000 kg

# Calculate for different activity levels
fig = go.Figure()

for activity in ['resting', 'moderate', 'intense']:
    oxygen_demands = []
    surface_areas = []

    for mass in body_masses:
        demand, area = predict_required_surface_area(mass, activity)
        oxygen_demands.append(demand)
        surface_areas.append(area)

    fig.add_trace(go.Scatter(
        x=body_masses,
        y=surface_areas,
        mode='lines',
        name=activity.capitalize(),
        line=dict(width=3)
    ))

# Add example animals
example_animals = {
    'Mouse': (0.02, 0.008),
    'Cat': (4, 0.4),
    'Human': (70, 70),
    'Horse': (500, 120),
    'Elephant': (5000, 400)
}

for animal, (mass, actual_area) in example_animals.items():
    fig.add_trace(go.Scatter(
        x=[mass],
        y=[actual_area],
        mode='markers+text',
        name=animal,
        text=[animal],
        textposition='top center',
        marker=dict(size=15, symbol='diamond'),
        showlegend=False
    ))

fig.update_layout(
    title='<b>Respiratory Surface Area Scaling with Body Mass</b>',
    xaxis_title='Body Mass (kg)',
    yaxis_title='Required Surface Area (m²)',
    xaxis_type='log',
    yaxis_type='log',
    height=500,
    hovermode='x unified'
)

fig.show()

print("\n📊 Key Insights from Scaling Analysis:")
print("=" * 60)
print("• Larger animals need proportionally MORE surface area")
print("• Activity level dramatically increases requirements")
print("• Why evolution favors thin membranes and large areas")
print("• Real animals (diamonds) match predicted requirements")
print("\nThis explains why whales have such enormous lung capacity!")


📊 Key Insights from Scaling Analysis:
• Larger animals need proportionally MORE surface area
• Activity level dramatically increases requirements
• Why evolution favors thin membranes and large areas
• Real animals (diamonds) match predicted requirements

This explains why whales have such enormous lung capacity!


## Part 5: Challenge Problems

Test your understanding with these design challenges:

### Challenge 1: Design a Respiratory System

**Scenario**: You're designing a respiratory system for a hypothetical aquatic vertebrate with these requirements:
- Body mass: 50 kg
- Lifestyle: Active predator (intense activity)
- Environment: Cold ocean water (10°C, 8 ml O₂/L)
- Constraint: Cannot exceed 5 m² total surface area

**Questions**:
1. What oxygen consumption rate must the system support?
2. Given the surface area constraint, how thin must the membrane be?
3. What concentration gradient would be needed?
4. Is this design realistic? Why or why not?

<details>
<summary>Click for hints</summary>

- Use the metabolic scaling function with 'intense' activity
- Remember temperature affects diffusion coefficient
- Compare your design to the shark gill preset
- Consider counter-current flow advantages
</details>

In [19]:
# Challenge 1 workspace
# Use this cell for your calculations

# Step 1: Calculate oxygen demand
body_mass = 50  # kg
oxygen_demand = calculate_oxygen_demand(body_mass, 'intense')
print(f"Oxygen Consumption Required: {oxygen_demand:.2f} ml O₂/min")

# Step 2: Design your system
surface_area = 5.0  # m² (maximum allowed)
gradient = 45  # mmHg (estimate for counter-current in cold water)

# What thickness is needed?
# Rearrange Fick's Law: T = (A × ΔP × D) / Rate
D_cold = 0.00023 * (1.3 ** ((10 - 37) / 10))  # Temperature correction
required_thickness = (surface_area * gradient * D_cold) / oxygen_demand

print(f"\nRequired membrane thickness: {required_thickness:.3f} μm")
print(f"\nComparison to shark gill: {respiratory_presets['Great White Shark Gill']['thickness']:.2f} μm")

# Is this realistic?
if required_thickness < 0.1:
    print("\n⚠ WARNING: Membrane too thin - structurally impossible!")
    print("Consider: increasing surface area or reducing activity level")
elif required_thickness > 2.0:
    print("\n⚠ WARNING: Membrane too thick - inefficient gas exchange!")
    print("Consider: increasing surface area or improving gradient")
else:
    print("\n✓ Design appears feasible! Similar to real fish gills.")

# Your analysis here:
print("\n" + "="*60)
print("YOUR ANALYSIS:")
print("="*60)
# Add your thoughts about the design

Oxygen Consumption Required: 2820.45 ml O₂/min

Required membrane thickness: 0.000 μm

Comparison to shark gill: 0.20 μm

⚠ WARNING: Membrane too thin - structurally impossible!
Consider: increasing surface area or reducing activity level

YOUR ANALYSIS:


### Challenge 2: High-Altitude Adaptation

**Scenario**: A bar-headed goose must maintain flight performance at 9,000 meters elevation where:
- Atmospheric pressure: 30% of sea level
- Available oxygen: reduced by 70%
- Flight requires intense metabolism

**Questions**:
1. How does the concentration gradient change at altitude?
2. What anatomical adaptations help compensate?
3. Calculate the exchange rate at altitude vs sea level
4. Why is unidirectional airflow crucial?

<details>
<summary>Click for hints</summary>

- Partial pressure decreases with altitude
- Enhanced hemoglobin increases effective gradient
- Thinner membranes and larger surface area help
- Bird air sacs eliminate dead space
</details>

In [20]:
# Challenge 2 workspace

# Load bar-headed goose data
goose_data = respiratory_presets['Bar-Headed Goose Air Sacs']

# Sea level conditions
gradient_sea_level = goose_data['gradient']  # mmHg
rate_sea_level = calculate_gas_exchange(
    goose_data['surface_area'],
    gradient_sea_level,
    goose_data['thickness'],
    temperature=10  # cold at altitude
)

print("SEA LEVEL PERFORMANCE:")
print(f"Gradient: {gradient_sea_level} mmHg")
print(f"Exchange Rate: {rate_sea_level:.2f} ml O₂/min")

# High altitude conditions (9,000m)
# Atmospheric pressure reduced to 30%
gradient_altitude = gradient_sea_level * 0.30  # Reduced oxygen

# But! Enhanced hemoglobin partially compensates
# Increases effective gradient by ~25%
gradient_altitude_enhanced = gradient_altitude * 1.25

rate_altitude = calculate_gas_exchange(
    goose_data['surface_area'],
    gradient_altitude_enhanced,
    goose_data['thickness'],
    temperature=10
)

print("\nHIGH ALTITUDE PERFORMANCE (9,000m):")
print(f"Base Gradient: {gradient_altitude:.2f} mmHg (70% reduction)")
print(f"Enhanced Gradient: {gradient_altitude_enhanced:.2f} mmHg (hemoglobin adaptation)")
print(f"Exchange Rate: {rate_altitude:.2f} ml O₂/min")

print("\n" + "="*60)
print(f"Performance Retention: {(rate_altitude/rate_sea_level)*100:.1f}%")
print("="*60)

print("\nKEY ADAPTATIONS:")
print("✓ Ultra-thin membrane (0.15 μm)")
print("✓ Enhanced hemoglobin oxygen affinity")
print("✓ Unidirectional airflow (no dead space)")
print("✓ Efficient counter-current-like exchange")

# Your analysis:
print("\nYOUR ANALYSIS:")
# Why is this level of performance remarkable?
# What would happen with mammalian lungs?

SEA LEVEL PERFORMANCE:
Gradient: 70 mmHg
Exchange Rate: 0.04 ml O₂/min

HIGH ALTITUDE PERFORMANCE (9,000m):
Base Gradient: 21.00 mmHg (70% reduction)
Enhanced Gradient: 26.25 mmHg (hemoglobin adaptation)
Exchange Rate: 0.02 ml O₂/min

Performance Retention: 37.5%

KEY ADAPTATIONS:
✓ Ultra-thin membrane (0.15 μm)
✓ Enhanced hemoglobin oxygen affinity
✓ Unidirectional airflow (no dead space)
✓ Efficient counter-current-like exchange

YOUR ANALYSIS:


### Challenge 3: Biomimetic Application

**Scenario**: Design an artificial gill for SCUBA divers inspired by fish gill principles.

**Requirements**:
- Extract enough O₂ from seawater to support one human
- Human resting VO₂: ~250 ml/min
- Seawater O₂: 6-8 ml/L
- Must be portable (realistic surface area)

**Questions**:
1. How much water must flow through per minute?
2. What surface area would be needed?
3. What are the engineering challenges?
4. Is this technology feasible with current materials?

<details>
<summary>Click for hints</summary>

- Extraction efficiency matters (sharks: 60-70%)
- Counter-current flow is essential
- Membrane must be gas-permeable but water-impermeable
- Compare required area to shark gills
</details>

In [21]:
# Challenge 3 workspace

# Human oxygen requirements
human_vo2 = 250  # ml O₂/min (resting)
seawater_o2 = 7  # ml O₂/L (average)

print("ARTIFICIAL GILL DESIGN CHALLENGE")
print("=" * 60)

# Step 1: Calculate water flow requirement
# Assuming 70% extraction efficiency (like shark)
extraction_efficiency = 0.70
required_flow = human_vo2 / (seawater_o2 * extraction_efficiency)

print(f"\nWater Flow Required: {required_flow:.2f} L/min")
print(f"That's {required_flow * 60:.0f} liters per hour!")

# Step 2: Calculate surface area needed
# Use Fick's Law with seawater parameters
gradient_seawater = 40  # mmHg (water-blood)
membrane_thickness = 0.3  # μm (feasible with modern materials)

# Rearrange: A = (Rate × T) / (ΔP × D)
D_seawater = 0.00023  # O₂ diffusion coefficient
required_area = (human_vo2 * membrane_thickness) / (gradient_seawater * D_seawater)

print(f"\nRequired Surface Area: {required_area:.2f} m²")
print(f"Compare to shark gill: {respiratory_presets['Great White Shark Gill']['surface_area']:.1f} m²")

# Step 3: Engineering challenges
print("\n" + "="*60)
print("ENGINEERING CHALLENGES:")
print("="*60)

challenges = [
    ("Water pumping", "Need powerful pump to move ~{}L/min".format(int(required_flow))),
    ("Surface area", "{}m² is very large - packaging challenge".format(required_area)),
    ("Membrane material", "Must be O₂-permeable but reject water and salt"),
    ("Biofouling", "Marine organisms would clog system rapidly"),
    ("Power source", "Pumping requires significant energy"),
    ("Pressure tolerance", "Must work at various diving depths")
]

for i, (challenge, description) in enumerate(challenges, 1):
    print(f"{i}. {challenge}: {description}")

print("\n" + "="*60)
print("FEASIBILITY ASSESSMENT:")
print("="*60)

if required_area > 10:
    feasibility = "Not currently feasible - too large"
elif required_flow > 100:
    feasibility = "Challenging - high flow rate"
else:
    feasibility = "Potentially feasible with advanced materials"

print(f"\nConclusion: {feasibility}")
print("\nNote: Current artificial gill prototypes work for a few minutes only.")
print("The physics tells us why fish gills are so beautifully optimized!")

# Your biomimetic design ideas:
print("\n" + "="*60)
print("YOUR BIOMIMETIC DESIGN IMPROVEMENTS:")
print("="*60)
# What innovations might help?
# What can we learn from fish gill structure?

ARTIFICIAL GILL DESIGN CHALLENGE

Water Flow Required: 51.02 L/min
That's 3061 liters per hour!

Required Surface Area: 8152.17 m²
Compare to shark gill: 7.0 m²

ENGINEERING CHALLENGES:
1. Water pumping: Need powerful pump to move ~51L/min
2. Surface area: 8152.173913043478m² is very large - packaging challenge
3. Membrane material: Must be O₂-permeable but reject water and salt
4. Biofouling: Marine organisms would clog system rapidly
5. Power source: Pumping requires significant energy
6. Pressure tolerance: Must work at various diving depths

FEASIBILITY ASSESSMENT:

Conclusion: Not currently feasible - too large

Note: Current artificial gill prototypes work for a few minutes only.
The physics tells us why fish gills are so beautifully optimized!

YOUR BIOMIMETIC DESIGN IMPROVEMENTS:


## Part 6: Export Your Work

Save your calculations and visualizations for your assignment or presentation:

In [22]:
# Export system
import os
from datetime import datetime

def export_lab_results():
    """
    Export comparison data and create summary report
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Create export directory
    export_dir = "/content"
    os.makedirs(export_dir, exist_ok=True)

    # Export comparison data if available
    if calculator.comparison_data:
        df = pd.DataFrame(calculator.comparison_data)
        csv_file = f"{export_dir}/lab_6_1_comparison_{timestamp}.csv"
        df.to_csv(csv_file, index=False)
        print(f"✓ Comparison data saved: {csv_file}")

    # Create summary visualization
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Respiratory System Comparison', 'Scaling Analysis'),
        vertical_spacing=0.15
    )

    if calculator.comparison_data:
        names = [d['name'] for d in calculator.comparison_data]
        rates = [d['exchange_rate'] for d in calculator.comparison_data]

        fig.add_trace(
            go.Bar(x=names, y=rates, marker_color='#2E86AB',
                  text=[f"{r:.1f}" for r in rates], textposition='auto'),
            row=1, col=1
        )

    # Add scaling curve
    masses = np.logspace(0, 3, 30)
    areas = [(calculate_oxygen_demand(m, 'resting') * 0.5) / (60 * 0.00023) for m in masses]

    fig.add_trace(
        go.Scatter(x=masses, y=areas, mode='lines', line=dict(width=3, color='#F18F01')),
        row=2, col=1
    )

    fig.update_xaxes(type='log', row=2, col=1, title_text='Body Mass (kg)')
    fig.update_yaxes(title_text='ml O₂/min', row=1, col=1)
    fig.update_yaxes(type='log', title_text='Surface Area (m²)', row=2, col=1)

    fig.update_layout(
        height=800,
        title_text=f"<b>Lab 6.1 Summary - {timestamp}</b>",
        showlegend=False
    )

    # Save as HTML (works better in Colab)
    html_file = f"{export_dir}/lab_6_1_summary_{timestamp}.html"
    fig.write_html(html_file)
    print(f"✓ Summary visualization saved: {html_file}")

    # Try to save as PNG (fallback)
    try:
        png_file = f"{export_dir}/lab_6_1_summary_{timestamp}.png"
        fig.write_image(png_file, width=1200, height=800)
        print(f"✓ PNG image saved: {png_file}")
    except:
        print("⚠ PNG export requires additional setup (kaleido)")
        print("  Use the HTML file instead, or run: pip install kaleido")

    print("\n" + "="*60)
    print("EXPORT COMPLETE!")
    print("="*60)
    print(f"Files saved to: {export_dir}")
    print("\nTo download:")
    print("1. Click the folder icon in the left sidebar")
    print("2. Navigate to /content")
    print("3. Right-click files and select 'Download'")

# Create export button
export_button = Button(
    description='📥 Export Results',
    button_style='success',
    icon='download',
    layout={'width': '200px', 'height': '40px'}
)
export_button.on_click(lambda b: export_lab_results())

display(HTML("<h3>Export Your Work</h3>"))
display(HTML("<p>Click below to save your comparison data and visualizations:</p>"))
display(export_button)

HTML(value='<h3>Export Your Work</h3>')

HTML(value='<p>Click below to save your comparison data and visualizations:</p>')

Button(button_style='success', description='📥 Export Results', icon='download', layout=Layout(height='40px', w…

## Summary & Reflection

### What You've Learned

Through this interactive lab, you've:
1. ✅ Applied Fick's Law to real respiratory systems
2. ✅ Compared gas exchange efficiency across vertebrate groups
3. ✅ Understood how anatomy determines respiratory performance
4. ✅ Analyzed metabolic scaling and its implications
5. ✅ Explored biomimetic applications

### Key Insights

- **Surface area is king**: The most effective way to increase gas exchange
- **Thin is in**: Membrane thickness critically limits exchange rate
- **Gradients matter**: Counter-current flow and ventilation maintain high gradients
- **Size scales predictably**: Larger animals need proportionally more surface area
- **Evolution optimizes**: Real respiratory systems balance multiple constraints

### Connection to Chapter 6

This lab reinforced concepts from:
- **Section 6.1**: Fick's Law and gas exchange imperatives
- **Section 6.2**: Gill efficiency and counter-current exchange
- **Section 6.3**: Lung evolution and surface area expansion
- **Section 6.4**: Bird respiratory supremacy
- **Section 6.6**: High-altitude adaptations

### Next Steps

Ready to explore more? Continue to:
- **Lab 6.2**: Counter-Current Exchange Simulator
- **Lab 6.3**: Respiratory Performance Across Environments

Or dive deeper into Chapter 6 to learn about diving adaptations, altitude physiology, and biomimetic technologies!

---

*This interactive lab is part of "A Pattern Hunter's Guide to Comparative Anatomy"*  
*Chapter 6: The Oxygen Revolution - Respiratory Systems*